In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("24120901_sf_fire_calls_App").getOrCreate()

24/12/09 16:35:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
from pyspark.sql.types import *

In [8]:
#sf_fire_file= "/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv" 
#fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)

NameError: name 'fire_schema' is not defined

In [13]:
fire_df= spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/sf-fire-calls.csv')

In [14]:
fire_df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [19]:
from pyspark.sql import functions as F

In [20]:
few_fire_df= (fire_df
    .select("IncidentNumber", "AvailableDtTm", "CallType")
    .where(F.col("CallType") != "Medical Incident"))
few_fire_df.show(5, truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [24]:
# 화제 신고로 기록된 calltype 종류 
#countDistinct()

(fire_df 
    .select("CallType")
    .where(F.col("CallType").isNotNull())
    .agg(F.countDistinct("CallType").alias("DistinctCallTypes"))
    .show())

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [26]:
#신고타입의 목록 NULL이 아닌 개별 CallType을 추출

(fire_df
    .select("CallType")
    .where(F.col("CallType").isNotNull())
    .distinct()
    .show(10,False))

+-----------------------------------+
|CallType                           |
+-----------------------------------+
|Elevator / Escalator Rescue        |
|Marine Fire                        |
|Aircraft Emergency                 |
|Confined Space / Structure Collapse|
|Administrative                     |
|Alarms                             |
|Odor (Strange / Unknown)           |
|Citizen Assist / Service Call      |
|HazMat                             |
|Watercraft in Distress             |
+-----------------------------------+
only showing top 10 rows



In [28]:
#칼럼 이름 변경 withColumnRenamed()
#Delay-> ResponseDelayedinMins 로변경

new_fire_df= fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")
(new_fire_df
    .select("ResponseDelayedinMins")
    .where(F.col("ResponseDelayedinMins")>5)
     .show(5, False))

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.35                 |
|6.25                 |
|5.2                  |
|5.6                  |
|7.25                 |
+---------------------+
only showing top 5 rows



In [54]:
#칼럼 타입 변경

fire_ts_df= (new_fire_df
    .withColumn("IncidentDate", F.to_timestamp(F.col("CallDate"), "MM/dd/yyyy"))
    .drop("CallDate")
    .withColumn("OnWatchDate", F.to_timestamp(F.col("WatchDate"), "MM/dd/yyyy"))
    .drop("WatchDate")
    .withColumn("AvailableDtTS", F.to_timestamp(F.col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ssa"))
    .drop("AvailableDtTm"))


In [57]:
fire_ts_df.columns

['CallNumber',
 'UnitID',
 'IncidentNumber',
 'CallType',
 'CallFinalDisposition',
 'Address',
 'City',
 'Zipcode',
 'Battalion',
 'StationArea',
 'Box',
 'OriginalPriority',
 'Priority',
 'FinalPriority',
 'ALSUnit',
 'CallTypeGroup',
 'NumAlarms',
 'UnitType',
 'UnitSequenceInCallDispatch',
 'FirePreventionDistrict',
 'SupervisorDistrict',
 'Neighborhood',
 'Location',
 'RowID',
 'ResponseDelayedinMins',
 'IncidentDate',
 'OnWatchDate']

In [60]:
fire_ts_df.filter(F.col("AvailableDtTm").isNull()).show(5, False)

+----------+------+--------------+----------------+--------------------+---------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+---------------------+-------------------------------------+-------------+---------------------+-------------------+-------------------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallFinalDisposition|Address                    |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood         |Location                             |RowID        |ResponseDelayedinMins|IncidentDate       |OnWatchDate        |
+----------+------+--------------+----------------+--------------------+---------------------------+----+-------+---------+-----------+----+------

In [56]:
(fire_ts_df
    .select("IncidentDate","OnWatchDate","AvailableDtTS")
    .show(5,False))

AnalysisException: cannot resolve '`AvailableDtTS`' given input columns: [ALSUnit, Address, Battalion, Box, CallFinalDisposition, CallNumber, CallType, CallTypeGroup, City, FinalPriority, FirePreventionDistrict, IncidentDate, IncidentNumber, Location, Neighborhood, NumAlarms, OnWatchDate, OriginalPriority, Priority, ResponseDelayedinMins, RowID, StationArea, SupervisorDistrict, UnitID, UnitSequenceInCallDispatch, UnitType, Zipcode];
'Project [IncidentDate#1174, OnWatchDate#1232, 'AvailableDtTS]
+- Project [CallNumber#88, UnitID#89, IncidentNumber#90, CallType#91, CallFinalDisposition#94, Address#96, City#97, Zipcode#98, Battalion#99, StationArea#100, Box#101, OriginalPriority#102, Priority#103, FinalPriority#104, ALSUnit#105, CallTypeGroup#106, NumAlarms#107, UnitType#108, UnitSequenceInCallDispatch#109, FirePreventionDistrict#110, SupervisorDistrict#111, Neighborhood#112, Location#113, RowID#114, ... 3 more fields]
   +- Project [CallNumber#88, UnitID#89, IncidentNumber#90, CallType#9

In [37]:
new_fire_df.select("CallDate").show(5)

+----------+
|  CallDate|
+----------+
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
+----------+
only showing top 5 rows



In [40]:
new_fire_df.select("OnWatchDate").show(5)

AnalysisException: cannot resolve '`OnWatchDate`' given input columns: [ALSUnit, Address, AvailableDtTm, Battalion, Box, CallDate, CallFinalDisposition, CallNumber, CallType, CallTypeGroup, City, FinalPriority, FirePreventionDistrict, IncidentNumber, Location, Neighborhood, NumAlarms, OriginalPriority, Priority, ResponseDelayedinMins, RowID, StationArea, SupervisorDistrict, UnitID, UnitSequenceInCallDispatch, UnitType, WatchDate, Zipcode];
'Project ['OnWatchDate]
+- Project [CallNumber#88, UnitID#89, IncidentNumber#90, CallType#91, CallDate#92, WatchDate#93, CallFinalDisposition#94, AvailableDtTm#95, Address#96, City#97, Zipcode#98, Battalion#99, StationArea#100, Box#101, OriginalPriority#102, Priority#103, FinalPriority#104, ALSUnit#105, CallTypeGroup#106, NumAlarms#107, UnitType#108, UnitSequenceInCallDispatch#109, FirePreventionDistrict#110, SupervisorDistrict#111, ... 4 more fields]
   +- Relation[CallNumber#88,UnitID#89,IncidentNumber#90,CallType#91,CallDate#92,WatchDate#93,CallFi

In [41]:
new_fire_df.select("AvailableDtTS").show(5)

AnalysisException: cannot resolve '`AvailableDtTS`' given input columns: [ALSUnit, Address, AvailableDtTm, Battalion, Box, CallDate, CallFinalDisposition, CallNumber, CallType, CallTypeGroup, City, FinalPriority, FirePreventionDistrict, IncidentNumber, Location, Neighborhood, NumAlarms, OriginalPriority, Priority, ResponseDelayedinMins, RowID, StationArea, SupervisorDistrict, UnitID, UnitSequenceInCallDispatch, UnitType, WatchDate, Zipcode];
'Project ['AvailableDtTS]
+- Project [CallNumber#88, UnitID#89, IncidentNumber#90, CallType#91, CallDate#92, WatchDate#93, CallFinalDisposition#94, AvailableDtTm#95, Address#96, City#97, Zipcode#98, Battalion#99, StationArea#100, Box#101, OriginalPriority#102, Priority#103, FinalPriority#104, ALSUnit#105, CallTypeGroup#106, NumAlarms#107, UnitType#108, UnitSequenceInCallDispatch#109, FirePreventionDistrict#110, SupervisorDistrict#111, ... 4 more fields]
   +- Relation[CallNumber#88,UnitID#89,IncidentNumber#90,CallType#91,CallDate#92,WatchDate#93,Ca

In [29]:
#가장 흔한 형태의 신고는?

(fire_ts_df
    .select("CallType")
    .where(F.col("CallType").isNotNull())
    .groupBy("CallType")
    .count()
    .orderBy("count", ascending=False)
    .show(n=10, truncate=False))

NameError: name 'fire_ts_df' is not defined

In [51]:
fire_df.filter(F.year(F.col("CallDate")) == 2018) .select("CallType") .where(F.col("CallType").isNotNull()).distinct() .show()

+--------+
|CallType|
+--------+
+--------+

